In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(autoload_with=engine)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
# Display the row's columns and data in dictionary format
first_row = session.query(Measurement).first()
first_row.__dict__

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
recent_date=session.query(Measurement.date).order_by(Measurement.date.desc()).first()[0]
recent_date

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results.

# Covert most recent date from string to 'date format'
recent_date=dt.datetime.strptime(recent_date, '%Y-%m-%d')

# Calculate 12 months ago
one_yr_ago = recent_date - dt.timedelta(days=365)

# Query to retreive precep data
precip_data = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= one_yr_ago).all()

# Store the results in a DF, with date sorted by most recent first
precip_df = pd.DataFrame(precip_data, columns=['Date', 'Precipitation']).sort_values(by='Date', ascending=False)
precip_df

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
plt.figure(figsize=(10, 6))
plt.bar(precip_df['Date'], precip_df['Precipitation'], label='Precipitation')
plt.ylabel('Precipitation (inches)')
plt.title('Precipitation Data for the Last 12 Months')
# set x-axis ticks to every nth date
n=360
plt.xticks(precip_df['Date'][::n],rotation =45)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
summary_stats = precip_df['Precipitation'].describe()
summary_stats

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number of stations in the dataset
total_stations = session.query(func.count(Station.station)).first()[0]
total_stations

In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.
station_counts = session.query(Measurement.station, func.count(Measurement.station)).\
                    group_by(Measurement.station).\
                    order_by(func.count(Measurement.station).desc()).all()

# Display the stations and their counts
print("Most Active Stations:")
print("----------------------")
for station, count in station_counts:
    print(f"Station ID: {station}, Count: {count}")

# Station ID USC00519281 has the greatest observations

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
most_active_station = 'USC00519281'

# Calculate the lowest, highest, and average temperature for the specified station
temperature_stats = session.query(func.min(Measurement.tobs),
                                  func.max(Measurement.tobs),
                                  func.avg(Measurement.tobs)).\
                    filter(Measurement.station == most_active_station).all()

# Display the temperature statistics
print(f"Temperature Statistics for Station {most_active_station}:")
print("---------------------------------------------")
print(f"Lowest Temperature: {temperature_stats[0][0]} F")
print(f"Highest Temperature: {temperature_stats[0][1]} F")
print(f"Average Temperature: {round(temperature_stats[0][2], 2)} F")

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram (12 bins)
most_active_station = 'USC00519281'

# Query to retrieve last twelve months of data
temperature_data = session.query(Measurement.date, Measurement.tobs).\
                    filter(Measurement.station == most_active_station).\
                    filter(Measurement.date >= one_yr_ago).all()

# Store the results in a DataFrame
temperature_df = pd.DataFrame(temperature_data, columns=['date', 'temperature'])
temperature_df['date'] = pd.to_datetime(temperature_df['date'])

# Plot the results as a histogram
plt.figure(figsize=(10, 6))
plt.hist(temperature_df['temperature'], bins=12, edgecolor='black')
plt.xlabel('Temperature (F)')
plt.ylabel('Frequency')
plt.title(f'Temperature Observations for Station {most_active_station}\nLast 12 Months')
plt.grid(axis='y', alpha=0.75)
plt.tight_layout()
plt.show()

# Close Session

In [ ]:
# Close Session
session.close()